<a href="https://colab.research.google.com/github/antoinevangorp/DatabasManagementPractice_IBT3205/blob/main/week11_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('lab_session.db')
c = conn.cursor()

In [2]:
commands = [
  """
    CREATE TABLE IF NOT EXISTS student (
      ID VARCHAR(5) PRIMARY KEY,
      name VARCAR(20),
      dept_name VARCAR(20),
      tot_cred DECIMAL(3,0)
    );
  """,
  """
    CREATE TABLE IF NOT EXISTS courses (
      ID INT PRIMARY KEY,
      name VARCHAR(255),
      year INT,
      semester VARCHAR(255),
      credits INT,
      maximum_grades INT
    );
  """,
  """
    CREATE TABLE IF NOT EXISTS takes (
      ID INT PRIMARY KEY,
      student_id INT,
      course_id INT,
      grade VARCHAR(2),
      FOREIGN KEY (student_id) REFERENCES student(ID),
      FOREIGN KEY (course_id) REFERENCES courses(ID)
    );
  """
  ]
data_to_insert = [
    """
      INSERT INTO student (ID, name, dept_name, tot_cred) VALUES
        (1, 'Alice', 'Computer Science', 12),
        (2, 'Bob', 'Mathematics', 31),
        (3, 'Charlie', 'Physics', 70),
        (4, 'David', 'Biology', 98),
        (5, 'Eve', 'Chemistry', 120);
    """,
    """
      INSERT INTO courses (ID, name, year, semester, credits, maximum_grades) VALUES
        (1, 'Math 101', 2008, 'Spring', 3, 5),
        (2, 'Physics 201', 2008, 'Spring', 4, 5),
        (3, 'History 101', 2008, 'Spring', 3, 5),
        (4, 'Computer Science 301', 2008, 'Spring', 4, 5),
        (5, 'English 101', 2008, 'Spring', 3, 5),
        (6, 'Biology 202', 2008, 'Spring', 4, 5),
        (7, 'Art 201', 2008, 'Spring', 3, 5),
        (8, 'Chemistry 301', 2008, 'Spring', 4, 5),
        (9, 'Psychology 101', 2008, 'Spring', 3, 5),
        (10, 'Economics 202', 2008, 'Spring', 4, 5);
    """,
    """
      INSERT INTO takes (ID, student_id, course_id, grade) VALUES
        (1, 1, 9, 'A+'),
        (2, 1, 10, 'A+'),
        (3, 3, 3, 'F'),
        (4, 3, 4, 'A'),
        (5, 2, 1, 'C');
    """
]

a= """
      INSERT INTO takes (ID, student_id, course_id, grade) VALUES
        (1, 1, 9, 'A'),
        (2, 1, 10, 'B+'),
        (3, 3, 3, 'F'),
        (4, 3, 1, 'B'),
        (5, 2, 1, 'C'),
        (6, 2, 2, 'D+');
    """

In [3]:
for command in commands :
  c.execute(command)
# Commit changes and close connection

for command in data_to_insert :
  c.execute(command)
# Commit changes and close connection
conn.commit()

In [4]:
status = """
  SELECT
    ID,
    name,
    CASE
      WHEN tot_cred < 30 THEN 'Freshman'
      WHEN tot_cred < 60 THEN 'Sophomore'
      WHEN tot_cred < 90 THEN 'Junior'
      WHEN tot_cred < 120 THEN 'Senior'
      ELSE 'Graduate'
    END AS status
  FROM student;
"""
status_df = pd.read_sql_query(status, conn)
status_df

,ID,name,status
0,1,Alice,Freshman
1,2,Bob,Sophomore
2,3,Charlie,Junior
3,4,David,Senior
4,5,Eve,Graduate


In [5]:
count = """
  SELECT
    CASE
      WHEN tot_cred < 30 THEN 'Freshman'
      WHEN tot_cred < 60 THEN 'Sophomore'
      WHEN tot_cred < 90 THEN 'Junior'
      WHEN tot_cred < 120 THEN 'Senior'
      ELSE 'Graduate'
    END AS status,
    COUNT(ID)
  FROM student
    GROUP BY status;
"""
count_df = pd.read_sql_query(count, conn)
count_df

,status,COUNT(ID)
0,Freshman,1
1,Graduate,1
2,Junior,1
3,Senior,1
4,Sophomore,1


In [6]:
test = """
SELECT
    s.ID AS student_id,
    s.name AS student_name,
    GPA,
    CASE
        WHEN GPA BETWEEN 3.75 AND 4.50 THEN 'A'
        WHEN GPA BETWEEN 3.00 AND 3.75 THEN 'B'
        WHEN GPA BETWEEN 2.00 AND 3.00 THEN 'C'
        WHEN GPA BETWEEN 1.50 AND 2.00 THEN 'D'
        ELSE 'F'
    END AS letter_grade
FROM (
    SELECT
        s.ID,
        s.name,
        SUM(5 * CASE
            WHEN t.grade = 'A+' THEN 4.50 * c.credits
            WHEN t.grade = 'A' THEN 4.00 * c.credits
            WHEN t.grade = 'B+' THEN 3.50 * c.credits
            WHEN t.grade = 'B' THEN 3.00 * c.credits
            WHEN t.grade = 'C+' THEN 2.50 * c.credits
            WHEN t.grade = 'C' THEN 2.00 * c.credits
            WHEN t.grade = 'D+' THEN 1.50 * c.credits
            WHEN t.grade = 'D' THEN 1.00 * c.credits
            WHEN t.grade = 'F+' THEN 0.50 * c.credits
            WHEN t.grade = 'F' THEN 0.00 * c.credits
            ELSE 0.00
          END) / SUM(c.credits * c.maximum_grades) GPA
    FROM
        student s
    JOIN
        takes t ON s.ID = t.student_id
    JOIN
        courses c ON t.course_id = c.ID
) AS gpa_table
JOIN
  student s
JOIN
  takes t ON s.ID = t.student_id
JOIN
  courses c ON t.course_id = c.ID

"""

test_df = pd.read_sql_query(test, conn)
test_df

,student_id,student_name,GPA,letter_grade
0,1,Alice,3.147059,B
1,1,Alice,3.147059,B
2,2,Bob,3.147059,B
3,3,Charlie,3.147059,B
4,3,Charlie,3.147059,B


In [7]:
grade = """
  SELECT
      s.ID AS student_id,
      s.name AS student_name,
      SUM(5 * CASE
          WHEN t.grade = 'A+' THEN 4.50 * c.credits
          WHEN t.grade = 'A' THEN 4.00 * c.credits
          WHEN t.grade = 'B+' THEN 3.50 * c.credits
          WHEN t.grade = 'B' THEN 3.00 * c.credits
          WHEN t.grade = 'C+' THEN 2.50 * c.credits
          WHEN t.grade = 'C' THEN 2.00 * c.credits
          WHEN t.grade = 'D+' THEN 1.50 * c.credits
          WHEN t.grade = 'D' THEN 1.00 * c.credits
          WHEN t.grade = 'F+' THEN 0.50 * c.credits
          WHEN t.grade = 'F' THEN 0.00 * c.credits
          ELSE 0.00
        END) / SUM(c.credits * c.maximum_grades) GPA
  FROM
      student s
  JOIN
      takes t ON s.ID = t.student_id
  JOIN
      courses c ON t.course_id = c.ID
  GROUP BY
      s.ID, s.name
"""
grade_df = pd.read_sql_query(grade, conn)
grade_df

,student_id,student_name,GPA
0,1,Alice,4.500000
1,2,Bob,2.000000
2,3,Charlie,2.285714
